# Load Data

In [5]:
import os
import sys
import pandas as pd

folder_path = os.path.join(os.path.dirname(os.getcwd()), 'Data_Test_Multi_Raw')
print(folder_path)
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

print(data_test.head())

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\Data_Test_Multi_Raw
   day  month  year  is_holiday    location  temperature
0   10      5  2022       False       Rehau         7.74
1   20      9  2020       False  Holzminden         9.37
2   12      5  2020       False    Grafenau         8.97
3    1      4  2020       False     Parchim        10.05
4   23      8  2022       False     Ansbach         7.32


In [6]:
import numpy as np

# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []
final_order_quantities_DT = []

# Parameters for multi-item newsvendor problem
prices = np.array([0.3, 0.5, 0.6, 0.5, 0.5, 0.5]) #price data
costs = np.array([0.06, 0.06, 0.06, 0.06, 0.06, 0.06]) #cost data
salvages = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01]) #salvage data
underage_data = prices - costs 
overage_data = costs - salvages 


alpha_data = np.array([             #alpha data
    [0.0, 0.1, 0.05, 0.1, 0.05, 0.1],
    [0.15, 0.0, 0.1, 0.05, 0.05, 0.05],
    [0.1, 0.2, 0.0, 0.05, 0.1, 0.05],
    [0.05, 0.05, 0.05, 0.0, 0.15, 0.2],
    [0.1, 0.05, 0.15, 0.2, 0.0, 0.05],
    [0.05, 0.1, 0.05, 0.15, 0.1, 0.0]
])


# Preprocessing

In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Define preprocessing for numeric columns (scale them)
numeric_features = ['temperature']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Define preprocessing for categorical features (encode them)
categorical_features = ['location']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)],
    remainder='passthrough')

# Preprocessing on train data
X_train = preprocessor.fit_transform(data_train)

# Preprocessing on test data
X_test = preprocessor.transform(data_test)


print(X_test)


[[-1.2614810567465224 0.0 0.0 ... 5 2022 False]
 [0.03038460101498815 0.0 0.0 ... 9 2020 False]
 [-0.2866376462884491 0.0 0.0 ... 5 2020 False]
 ...
 [-1.2614810567465224 0.0 0.0 ... 1 2021 False]
 [-1.0316399274515304 0.0 0.0 ... 1 2022 False]
 [0.03038460101498815 0.0 0.0 ... 4 2020 True]]


# ANN

In [169]:
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasRegressor
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
from scipy.stats import reciprocal
from sklearn.model_selection import KFold 

In [ ]:
def make_nvps_loss(alpha, u, o):

    # transofrm the alpha, u, o to tensors
    u = tf.convert_to_tensor(underage_data, dtype=tf.float32) #underage costs
    o = tf.convert_to_tensor(overage_data, dtype=tf.float32) #overage costs
    alpha = tf.convert_to_tensor(alpha_data, dtype=tf.float32) #substitution matrix

    # define the loss function
    @tf.autograph.experimental.do_not_convert
    def nvps_loss(y_true, y_pred):
        q = tf.maximum(y_pred, 0.)

        # Calculate the demand increase for each product due to substitutions from other products
        demand_increase = tf.matmul( tf.maximum(0.0, y_true - y_pred),alpha)
        # Adjusted demand is the original demand plus the increase due to substitutions
        adjusted_demand = y_true + demand_increase

        profits = tf.matmul(q,tf.transpose(u)) - tf.matmul(tf.maximum(0.0,q - adjusted_demand), tf.transpose(u+o))

        return -tf.math.reduce_mean(profits)
    
    return nvps_loss

In [170]:
def loss_complex(y_true, y_pred):

    # Cast numpy arrays to tensors
    u = tf.convert_to_tensor(underage_data, dtype=tf.float32) #underage costs
    o = tf.convert_to_tensor(overage_data, dtype=tf.float32) #overage costs
    alpha = tf.convert_to_tensor(alpha_data, dtype=tf.float32) #substitution matrix

    # Cast y_true to float32
    y_true = tf.cast(y_true, dtype=tf.float32)

    # Calculate the demand increase for each product due to substitutions from other products
    demand_increase = tf.matmul( tf.maximum(0.0, y_true - y_pred),alpha)

    # Adjusted demand is the original demand plus the increase due to substitutions
    adjusted_demand = y_true + demand_increase

    # Compute the loss with adjusted demand
    loss = -tf.reduce_mean(u * y_pred - (u + o) * tf.maximum(y_pred - adjusted_demand, 0))
    return loss

from keras.utils import get_custom_objects
get_custom_objects().update({'loss_complex': loss_complex})

# Model creation function 
def create_model(n_hidden, n_neurons, learning_rate, activation):
    model = Sequential()
    # Input Layer
    model.add(Dense(n_neurons,input_dim=15, activation=activation))
    # Hidden Layer
    for _ in range(n_hidden):
        model.add(Dense(n_neurons, activation=activation))
    # Output Layer
    model.add(Dense(6))
    model.compile(loss= loss_complex,
                  optimizer=Adam(learning_rate=learning_rate))
    return model

# Model builder function 
def model_builder(n_hidden, n_neurons, learning_rate, activation, batch_size, epochs):
    return KerasRegressor(build_fn=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons, 
                          learning_rate=learning_rate, activation=activation, batch_size=batch_size, epochs=epochs)

# Create a baseline model
model_ANN = model_builder(1,30,3e-3,'relu', 32, 20)


# Define the parameter grid
from scipy.stats import reciprocal
params = {
    "n_hidden": range(0, 15),
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 1e-2),
    "batch_size": [16, 32, 64, 128],
    "epochs": [10,15, 20, 25, 30],
    "activation": ['relu', 'sigmoid', 'tanh'] #
}

# Optimize the model using RandomizedSearchCV
rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distributions=params, cv=KFold(10))

# Fit the model
rnd_search_cv_ANN.fit(X_train, target_train)
print(rnd_search_cv_ANN.best_params_)


c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'relu', 'batch_size': 16, 'epochs': 25, 'learning_rate': 0.00014593446517329207, 'n_hidden': 11, 'n_neurons': 27}


In [171]:
# Predict with best model
best_model_ANN = rnd_search_cv_ANN.best_estimator_
best_model_ANN.fit(X_train, target_train)
target_pred_ANN = best_model_ANN.predict(X_test)

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


KerasRegressor(
	model=None
	build_fn=<function create_model at 0x000001C0FC33F920>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=16
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=25
	n_hidden=11
	n_neurons=27
	learning_rate=0.00014593446517329207
	activation=relu
)


# DT

In [25]:
import argparse
from typing import Dict, List, Tuple

import numpy as np
from matplotlib import pyplot as plt

import xgboost as xgb


def custom_rmse_model(plot_result: bool, strategy: str) -> None:
    """Train using Python implementation of Squared Error."""

    # As the experimental support status, custom objective doesn't support matrix as
    # gradient and hessian, which will be changed in future release.
    def gradient(predt: np.ndarray, dtrain: xgb.DMatrix) -> np.ndarray:
        """Compute the gradient squared error."""
        y = dtrain.get_label().reshape(predt.shape)
        return (predt - y).reshape(y.size)

    def hessian(predt: np.ndarray, dtrain: xgb.DMatrix) -> np.ndarray:
        """Compute the hessian for squared error."""
        return np.ones(predt.shape).reshape(predt.size)

    def squared_log(
        predt: np.ndarray, dtrain: xgb.DMatrix
    ) -> Tuple[np.ndarray, np.ndarray]:
        grad = gradient(predt, dtrain)
        hess = hessian(predt, dtrain)
        return grad, hess

    def rmse(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
        y = dtrain.get_label().reshape(predt.shape)
        v = np.sqrt(np.sum(np.power(y - predt, 2)))
        return "PyRMSE", v

    X, y = X_train, target_train
    print(X.shape, y.shape)

    Xy = xgb.DMatrix(X, y)
    results: Dict[str, Dict[str, List[float]]] = {}
    # Make sure the `num_target` is passed to XGBoost when custom objective is used.
    # When builtin objective is used, XGBoost can figure out the number of targets
    # automatically.
    booster = xgb.train(
        {
            "tree_method": "hist",
            "num_target": y.shape[1],
            "multi_strategy": strategy,
        },
        dtrain=Xy,
        num_boost_round=128,
        obj=squared_log,
        evals=[(Xy, "Train")],
        evals_result=results,
        custom_metric=rmse,
    )

    y_predt = booster.inplace_predict(X)
    print(y_predt)



custom_rmse_model(1, "multi_output_tree")

(800, 15) (800, 6)
[0]	Train-rmse:39.70493	Train-PyRMSE:2750.83813
[1]	Train-rmse:27.91991	Train-PyRMSE:1934.34839
[2]	Train-rmse:19.65481	Train-PyRMSE:1361.72498
[3]	Train-rmse:13.83080	Train-PyRMSE:958.22595
[4]	Train-rmse:9.74766	Train-PyRMSE:675.33740
[5]	Train-rmse:6.87293	Train-PyRMSE:476.17065
[6]	Train-rmse:4.85215	Train-PyRMSE:336.16669
[7]	Train-rmse:3.43304	Train-PyRMSE:237.84799
[8]	Train-rmse:2.43472	Train-PyRMSE:168.68213
[9]	Train-rmse:1.73343	Train-PyRMSE:120.09528
[10]	Train-rmse:1.23752	Train-PyRMSE:85.73800
[11]	Train-rmse:0.88808	Train-PyRMSE:61.52793
[12]	Train-rmse:0.64395	Train-PyRMSE:44.61405
[13]	Train-rmse:0.47200	Train-PyRMSE:32.70147
[14]	Train-rmse:0.35132	Train-PyRMSE:24.34045
[15]	Train-rmse:0.26536	Train-PyRMSE:18.38495
[16]	Train-rmse:0.20393	Train-PyRMSE:14.12897
[17]	Train-rmse:0.15983	Train-PyRMSE:11.07327
[18]	Train-rmse:0.12712	Train-PyRMSE:8.80738
[19]	Train-rmse:0.10267	Train-PyRMSE:7.11298
[20]	Train-rmse:0.08387	Train-PyRMSE:5.81069
[21]	Train-

In [22]:
import numpy as np
import xgboost as xgb

def gradient(predt, dtrain):
    y_true = dtrain.get_label()
    y_pred = predt

    # Calculate the demand increase for each product due to substitutions from other products
    demand_increase = np.dot(np.maximum(0.0, y_true - y_pred), alpha_data)
    # Adjusted demand is the original demand plus the increase due to substitutions
    adjusted_demand = y_true + demand_increase

    q = np.maximum(y_pred, 0.)
    profits = np.dot(q, underage_data.T) - np.dot(np.maximum(0.0, q - adjusted_demand), (underage_data + overage_data).T)

    # Calculate gradient
    grad = -np.mean(profits)

    return grad.reshape(y_true.size)

def hessian(predt, dtrain):
    y_true = dtrain.get_label()
    # Return a matrix of ones with the same shape as y_true
    return np.ones_like(y_true).reshape(y_true.size)

def nvps_obj(predt, dtrain):
    grad = gradient(predt, dtrain)
    hess = hessian(predt, dtrain)
    return grad, hess

# Create a DMatrix from your training data
dtrain = xgb.DMatrix(X_train, label=target_train)

# Define the parameters for the xgb.train function
params = {
    'tree_method': 'hist',
    'num_target': 6,
    'multi_strategy': 'multi_output_tree',
    'n_boost_round': 128,
    'learning_rate': 0.1,
    'random_state': 42
}

# Train the model
model = xgb.train(params, dtrain, obj=nvps_obj)

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\xgboost\core.py:160: UserWarning: [15:35:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)


ValueError: operands could not be broadcast together with shapes (4800,) (800,6) 

In [19]:
import numpy as np


def nvps_obj(y_pred, y_true):
    # Convert inputs to numpy arrays
    print('true: ' + str(np.shape(y_true)))
    print('pred: ' + str(np.shape(y_pred)))
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate the demand increase for each product due to substitutions from other products
    demand_increase = np.dot(np.maximum(0.0, y_true - y_pred), alpha_data)
    # Adjusted demand is the original demand plus the increase due to substitutions
    adjusted_demand = y_true + demand_increase

    q = np.maximum(y_pred, 0.)
    profits = np.dot(q, underage_data.T) - np.dot(np.maximum(0.0, q - adjusted_demand), (underage_data + overage_data).T)

    # Calculate gradient and hessian
    grad = -np.mean(profits)
    hess = np.ones_like(y_true)

    return grad, hess


from xgboost import XGBRegressor

model = XGBRegressor(objective=nvps_obj, n_estimators=100, learning_rate=0.1, random_state=42, 
                         num_target = 6, tree_method='hist', multi_strategy="multi_output_tree")

model.fit(X_train, target_train)

true: (800, 6)
pred: (4800,)


ValueError: operands could not be broadcast together with shapes (800,6) (4800,) 

In [12]:
print(np.shape(target_train))

(800, 6)


true: (800, 6)
pred: (4800,)


ValueError: operands could not be broadcast together with shapes (800,6) (4800,) 

In [172]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.multioutput import MultiOutputRegressor

# Specify the parameter grid for RandomizedSearchCV
param_distribs = {
    'estimator__boosting_type': ['gbdt', 'dart', 'goss'],
    'estimator__num_leaves': [10, 20, 30, 40, 50],
    'estimator__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'estimator__feature_fraction': [0.8, 0.9, 1.0],
    'estimator__bagging_fraction': [0.8, 0.9, 1.0],
    'estimator__bagging_freq': [3, 4, 5, 6, 7],
}

critical_ratio = np.mean(underage_data) / (np.mean(overage_data) + np.mean(underage_data))
cr = critical_ratio

# Create the LightGBM model
model = lgb.LGBMRegressor(objective='quantile', alpha=cr, metric='quantile', verbose=0)

# Wrap the model with MultiOutputRegressor
model = MultiOutputRegressor(model)

# Perform RandomizedSearchCV
rnd_search = RandomizedSearchCV(model, param_distributions=param_distribs, n_iter=10, cv=3)

# Fit the model
rnd_search.fit(X_train, target_train)
print(rnd_search.best_params_)



[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
6 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\la

In [173]:
# Get the best model
best_model = rnd_search.best_estimator_

# Predict the demand using the best model
target_pred_DT = best_model.predict(X_test)
target_pred_DT = pd.DataFrame(np.row_stack(target_pred_DT))
print(target_pred_DT)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=1.0, subsample=1.0 will be ignored. Current value: bagging_fraction=1.0
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 

# Costs

In [174]:
# Loop over each week in target_test
overall_costs_ANN = 0
overall_costs_DT = 0

for i in range(len(target_test)):
    for j in range(len(target_test.columns)):
        # Calculate understock and overstock costs
        cost_ANN = 0
        cost_DT = 0

        if target_pred_ANN.iloc[i, j] < target_test.iloc[i, j]:
            cost_ANN = (prices[j] - costs[j]) * (target_test.iloc[i, j] - np.round(target_pred_ANN.iloc[i, j]))

        if target_pred_ANN.iloc[i, j] > target_test.iloc[i, j]:
            cost_ANN = (costs[j] - salvages[j]) * (np.round(target_pred_ANN.iloc[i, j]) - target_test.iloc[i, j])
        
        if target_pred_DT.iloc[i, j] < target_test.iloc[i, j]:
            cost_DT = (prices[j] - costs[j]) * (target_test.iloc[i, j] - np.round(target_pred_DT.iloc[i, j]))

        if target_pred_DT.iloc[i, j] > target_test.iloc[i, j]:
            cost_DT = (costs[j] - salvages[j]) * (np.round(target_pred_DT.iloc[i, j]) - target_test.iloc[i, j])
        
        # Calculate the total costs for the week
        overall_costs_ANN += cost_ANN
        overall_costs_DT += cost_DT

# Print the overall costs
print('Overall costs for ANN: ', int(overall_costs_ANN))
print('Overall costs for DT: ', int(overall_costs_DT))

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'